# Search for collections with CMR search

In [35]:
import requests
from xml.dom.minidom import parse, parseString
def prettyprint(response):
    pretty = parseString(response.content).toprettyxml()
    print(pretty)
def printhits(response):
    print(response.headers['CMR-Hits'] + " hits")

## Find all collections
This request returns all collections with no filtering.

In [36]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections")
printhits(response)

28347 hits


## Find a collection by concept id
We can add a concept id constraint to the search to get the collection matching that concept id.

A CMR concept id is in the format `<concept-type-prefix> <unique-number> "-" <provider-id>`

* `concept-type-prefix` is a single capital letter prefix indicating the concept type. "C" is used for collections
* `unique-number` is a single number assigned by the CMR during ingest.
* `provider-id` is the short name for the provider. i.e. "LPDAAC_ECS"

In [37]:
content_id = "C1934541400-SCIOPS"
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?concept_id=" + content_id)
print("The following collection has the content_id " + content_id + ":\n")
prettyprint(response)

The following collection has the content_id C1934541400-SCIOPS:

<?xml version="1.0" ?>
<results>
	<hits>1</hits>
	<took>9</took>
	<references>
		<reference>
			<name>&quot;The Omnivores Dilemma&quot;: The Effect of Autumn Diet on Winter Physiology and Condition of Juvenile Antarctic Krill</name>
			<id>C1934541400-SCIOPS</id>
			<location>https://cmr.earthdata.nasa.gov:443/search/concepts/C1934541400-SCIOPS/1</location>
			<revision-id>1</revision-id>
		</reference>
	</references>
</results>



## Find collections by doi value
Find a collection matching a collection doi value. Note more than one doi value may be supplied.

In [38]:
doi = "*"
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?doi=" + doi)
printhits(response)

0 hits


In [39]:
doi1 = "1*"
doi2 = "2*"
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?doi=" + doi1 + "&doi=" + doi2)
printhits(response)

0 hits


## Find collections by echo collection id 
Find a collection matching a echo collection id. Note more than one echo collection id may be supplied.

0 hits


In [ ]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?echo_collection_id=C1000000001-CMR_PROV2")
printhits(response)

## <a name="c-provider-short-name"></a> Find collections by provider short name

This searches for collections whose provider matches the given provider short names. This supports `ignore_case` option, but not the `pattern` option.

In [41]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?provider_short_name=SHORT_5&options[provider_short_name][ignore_case]=true")
printhits(response)

0 hits


## Find collections by entry title

One entry title

In [42]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?entry_title=DatasetId%204")
printhits(response)

0 hits


a dataset id (alias for entry title)

In [43]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?dataset_id=DatasetId%204")
printhits(response)

0 hits


with multiple dataset ids

In [44]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?entry_title=DatasetId%204&entry_title=DatasetId%205")
printhits(response)

0 hits


with a entry title case insensitively

In [45]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?entry_title=datasetId%204&options[entry_title][ignore_case]=true")
printhits(response)

0 hits


with a entry title pattern


In [46]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?entry_title=DatasetId*&options[entry_title][pattern]=true")
printhits(response)

0 hits


## Find collections by entry id

One entry id

In [47]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?entry_id=SHORT_V5")
printhits(response)

0 hits


## Find collections by archive center

This supports `pattern` and `ignore_case`.

Find collections matching 'archive_center' param value

In [48]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?archive_center=LARC")
printhits(response)

33 hits


In [49]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?archive_center=Sedac+AC")
printhits(response)

0 hits


Find collections matching any of the 'archive_center' param values


In [50]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?archive_center=Larc&archive_center=SEDAC")
printhits(response)

291 hits


## Find collections by data center

This supports `pattern`, `and`, and `ignore_case`.

Find collections matching 'data_center' param value

In [51]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?data_center=LARC")
printhits(response)

33 hits


In [52]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?data_center=Sedac+AC")
printhits(response)

0 hits


Find collections matching any of the 'data_center' param values

In [53]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?data_center=Larc&data_center=SEDAC")
printhits(response)

291 hits


## Find collections with temporal

The temporal datetime has to be in yyyy-MM-ddTHH:mm:ssZ format.

In [54]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?temporal=2000-01-01T10:00:00Z,2010-03-10T12:00:00Z,30,60&temporal=2000-01-01T10:00:00Z,,30&temporal=2000-01-01T10:00:00Z,2010-03-10T12:00:00Z")
printhits(response)

20998 hits


The first two values of the parameter together define the temporal bounds. See under [Temporal Range searches](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#temporal-range-searches) for different ways of specifying the temporal bounds including ISO 8601.

For temporal range search, the default is inclusive on the range boundaries. This can be changed by specifying `exclude_boundary` option with `options[temporal][exclude_boundary]=true`. This option has no impact on periodic temporal searches.

The collection's temporal range or the temporal range of the granules in the collection can be searched. `options[temporal][limit_to_granules]=true` will indicate that the temporal search should find collections based on the minimum and maximum values of each collection's granules' temporal range. If a collection does not have any granules it will search the collection's temporal range.

If a temporal range search is performed, the search results will be sorted by the temporal overlap across all ranges provided, with usage score being the tie-breaker. If a keyword search is performed in conjunction with the temporal range search, search results are first sorted by relevancy score, then by temporal overlap, then usage score. If a keyword search is used in conjuction with usage-score sort key, the usage-score will be used instead of relevancy score.

## Find collections by project

Note: An alias for the parameter 'project' is 'campaign'. As such 'campaign' can be used in place of 'project'.

This supports `pattern`, `ignore_case` and option `and`.

Find collections matching 'project' param value

In [55]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?project=ESI")
printhits(response)

4 hits


Find collections matching any of the 'project' param values

In [56]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?project=ESI&project=EVI&project=EPI")
printhits(response)

12 hits


Find collections that match all of the 'project' param values


In [57]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?project=ESI&project=EVI&project=EPI&options[project][and]=true")
printhits(response)

0 hits


## Find collections by updated_since

  Find collections which have revision date starting at or after 'updated_since' param value

In [58]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?updated_since=2014-05-08T20:06:38.331Z")
printhits(response)

28347 hits


## Find collections by created_at

 This supports option `and`.

 Find collections which were created within the ranges of datetimes. The datetime has to be in yyyy-MM-ddTHH:mm:ssZ format. The default is inclusive on the range boundaries.

In [59]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?created_at=2000-01-01T10:00:00Z,2010-03-10T12:00:00Z&created_at=2015-01-01T10:00:00Z,")
printhits(response)

28111 hits


## Find collections with new granules

  This supports option `and`.

  Find collections containing granules added within the range of datetimes. The datetime has to be in yyyy-MM-ddTHH:mm:ssZ format. The default is inclusive on the range boundaries.

In [60]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?has_granules_created_at=2015-01-01T10:00:00Z")
printhits(response)

7707 hits


## Find collections with granules revised inside of a given range

  This supports option `and`.

  Find collections containing granules created or updated within the range of datetimes. The datetime has to be in yyyy-MM-ddTHH:mm:ssZ format. The default is inclusive on the range boundaries.

In [61]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?has_granules_revised_at=2015-01-01T10:00:00Z,")
printhits(response)

7710 hits


## Find collections by revision_date

  This supports option `and`.

  Find collections which have revision date within the ranges of datetimes. The datetime has to be in yyyy-MM-ddTHH:mm:ssZ format. The default is inclusive on the range boundaries.

In [62]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?revision_date=2000-01-01T10:00:00Z,2010-03-10T12:00:00Z&revision_date=2015-01-01T10:00:00Z,")
printhits(response)

28347 hits


## Find collections by processing\_level\_id

This supports `pattern` and `ignore_case`.

Find collections matching 'processing_level_id'

In [63]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?processing_level_id=1B")
printhits(response)

473 hits


Find collections matching any of the 'processing\_level\_id' param values

In [64]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?processing_level_id=1B&processing_level_id=2B")
printhits(response)

475 hits


The alias 'processing_level' also works for searching by processing level id.

## Find collections by platform

This supports `pattern`, `ignore_case` and option `and`.

Find collections matching 'platform' param value

In [65]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?platform=1B")
printhits(response)

0 hits


Find collections matching any of the 'platform' param values

In [66]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?platform=1B&platform=2B")
printhits(response)

0 hits


## Find collections by instrument

This supports `pattern`, `ignore_case` and option `and`.

Find collections matching 'instrument' param value

In [67]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?instrument=1B")
printhits(response)

0 hits


Find collections matching any of the 'instrument' param values

In [68]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?instrument=1B&instrument=2B")
printhits(response)

0 hits


## Find collections by sensor.

Sensor search is deprecated and should be replaced with instrument. Sensors are now child instruments on an instrument.

This supports `pattern`, `ignore_case` and option `and`.

Find collections matching 'sensor' param value

In [69]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?sensor=1B")
printhits(response)

0 hits


Find collections matching any of the 'sensor' param values

In [70]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?sensor=1B&sensor=2B")
printhits(response)

0 hits


## Find collections by spatial\_keyword

This supports `pattern`, `ignore_case` and option `and`.

Find collections matching any of the 'spatial_keyword' param values

In [71]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?spatial_keyword=DC&spatial_keyword=LA")
printhits(response)

1 hits


## Find collections by Spatial

### Polygon

Polygon points are provided in counter-clockwise order. The last point should match the first point to close the polygon. The values are listed comma separated in longitude latitude order, i.e. lon1, lat1, lon2, lat2, lon3, lat3, and so on.
This parameter supports the and/or option as shown below.

In [77]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?polygon[]=10,10,30,10,30,20,10,20,10,10")
printhits(response)

7789 hits


### Bounding Box

Bounding boxes define an area on the earth aligned with longitude and latitude. The Bounding box parameters must be 4 comma-separated numbers: lower left longitude, lower left latitude, upper right longitude, upper right latitude.
This parameter supports the and/or option as shown below.

In [78]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?bounding_box[]=-10,-5,10,5")
printhits(response)

response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?bounding_box[]=-10,-5,10,5&bounding_box[]=-11,-6,11,6&options[bounding_box][or]=true")
printhits(response)

7812 hits
7824 hits


### Point

Search using a point involves using a pair of values representing the point coordinates as parameters. The first value is the longitude and second value is the latitude. This parameter supports the and/or option as shown below.


In [79]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?point=100,20")
printhits(response)

response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?point=100,20&point=80,20&options[point][or]=true")
printhits(response)

7649 hits
7683 hits


### Line

Lines are provided as a list of comma separated values representing coordinates of points along the line. The coordinates are listed in the format lon1, lat1, lon2, lat2, lon3, lat3, and so on. This parameter supports the and/or option as shown below.

In [80]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?line[]=-0.37,-14.07,4.75,1.27,25.13,-15.51")
printhits(response)

response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?line[]=-0.37,-14.07,4.75,1.27,25.13,-15.51&line[]=-1.37,-15.07,5.75,2.27,26.13,-16.51&options[line][or]=true")
printhits(response)

7812 hits
7839 hits


### Circle

Circle defines a circle area on the earth with a center point and a radius. The center parameters must be 3 comma-separated numbers: longitude of the center point, latitude of the center point, radius of the circle in meters. The circle center cannot be on North or South pole. The radius of the circle must be between 10 and 6,000,000.
This parameter supports the and/or option as shown below.

In [81]:
esponse = requests.get("https://cmr.earthdata.nasa.gov/search/collections?circle[]=-87.629717,41.878112,1000")
printhits(response)

response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?circle[]=-87.629717,41.878112,1000&circle[]=-75,41.878112,1000&options[circle][or]=true")
printhits(response)

7839 hits
8903 hits


Note: A query could consist of multiple spatial parameters of different types, two bounding boxes and a polygon for example. If multiple spatial parameters are present, all the parameters irrespective of their type are AND'd in a query. So, if a query contains two bounding boxes and a polygon for example, it will return only those collections which intersect both the bounding boxes and the polygon.